# Data Extraction ⬇️

In [1]:
import pandas as pd
import numpy as np
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
import plotly.express as px
import plotly.io as pio
from sklearn.discriminant_analysis import StandardScaler
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier

pio.renderers.default = "browser"

In [2]:
original_dataset = pd.read_csv('base_telecomx.csv')
original_dataset.head()

,Churn,customer.gender,customer.SeniorCitizen,customer.Partner,customer.Dependents,customer.tenure,account.Contract,account.PaymentMethod,account.Charges.Total,daily_bill
0,0,0,0,1,1,9.0,One year,Mailed check,593.30,2.20
1,0,1,0,0,0,9.0,Month-to-month,Mailed check,542.40,2.01
2,1,1,0,0,0,4.0,Month-to-month,Electronic check,280.85,2.34
3,1,1,1,1,0,13.0,Month-to-month,Electronic check,1237.85,3.17
4,1,0,1,1,0,3.0,Month-to-month,Mailed check,267.40,2.97


# Statistical Modeling 📊

In [3]:
x = original_dataset.drop('Churn', axis=1)
y = original_dataset['Churn']

colunas = x.columns

## Data encoding

In [4]:
dataset_onehot = make_column_transformer(
    (OneHotEncoder(drop = 'if_binary'), ['account.Contract', 'account.PaymentMethod']),
    remainder = 'passthrough',
    sparse_threshold = 0
)

x = dataset_onehot.fit_transform(x)
dataset_onehot.get_feature_names_out(colunas)

df_train_trans = pd.DataFrame(x, columns = dataset_onehot.get_feature_names_out(colunas))
df_train_trans.head()

,onehotencoder__account.Contract_Month-to-month,onehotencoder__account.Contract_One year,onehotencoder__account.Contract_Two year,onehotencoder__account.PaymentMethod_Bank transfer (automatic),onehotencoder__account.PaymentMethod_Credit card (automatic),onehotencoder__account.PaymentMethod_Electronic check,onehotencoder__account.PaymentMethod_Mailed check,remainder__customer.gender,remainder__customer.SeniorCitizen,remainder__customer.Partner,remainder__customer.Dependents,remainder__customer.tenure,remainder__account.Charges.Total,remainder__daily_bill
0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,9.0,593.30,2.20
1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,9.0,542.40,2.01
2,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,4.0,280.85,2.34
3,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,13.0,1237.85,3.17
4,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,3.0,267.40,2.97


In [5]:
# Renaming columns for better readability
df_train_trans.columns = [c.replace('onehotencoder__', '') for c in df_train_trans.columns]
df_train_trans.columns = [c.replace('(automatic)', '') for c in df_train_trans.columns]
df_train_trans.columns = [c.replace('remainder__', '') for c in df_train_trans.columns]
df_x_train = df_train_trans.copy()
df_x_train.head()

,account.Contract_Month-to-month,account.Contract_One year,account.Contract_Two year,account.PaymentMethod_Bank transfer,account.PaymentMethod_Credit card,account.PaymentMethod_Electronic check,account.PaymentMethod_Mailed check,customer.gender,customer.SeniorCitizen,customer.Partner,customer.Dependents,customer.tenure,account.Charges.Total,daily_bill
0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,9.0,593.30,2.20
1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,9.0,542.40,2.01
2,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,4.0,280.85,2.34
3,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,13.0,1237.85,3.17
4,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,3.0,267.40,2.97


## Churn Exploratory Analysis

In [6]:
df_y_train = y.reset_index(drop=True).to_frame()
df_y_train['0'] = df_y_train['Churn']
df_y_train.drop('0', axis=1, inplace=True)
df_y_train.head()

y_churn = df_y_train['Churn'].value_counts()[1]
y_n_churn = df_y_train['Churn'].value_counts()[0]

px.bar(x = ['Churn', 'No Churn'],
       y = [y_churn, y_n_churn],
       title = 'Churn vs No Churn',
       labels = {'x': 'Customer Status', 'y': 'Number of Customers'})


## balancing variables

In [7]:
df_x_train.value_counts(subset=None, normalize=False, sort=True, ascending=False, dropna=True)

account.Contract_Month-to-month  account.Contract_One year  account.Contract_Two year  account.PaymentMethod_Bank transfer   account.PaymentMethod_Credit card   account.PaymentMethod_Electronic check  account.PaymentMethod_Mailed check  customer.gender  customer.SeniorCitizen  customer.Partner  customer.Dependents  customer.tenure  account.Charges.Total  daily_bill
1.0                              0.0                        0.0                        0.0                                   0.0                                 0.0                                     1.0                                 1.0              0.0                     0.0               0.0                  1.0              20.20                  0.67          6
                                                                                                                                                                                                                                             0.0              0.0  

In [8]:
df_x_train['daily_bill'] = np.log1p(df_x_train['daily_bill'])
df_x_train['account.Charges.Total'] = np.log1p(df_x_train['account.Charges.Total'])

std_scaler = StandardScaler()
mm_scaler = MinMaxScaler()

df_x_train['daily_bill'] = std_scaler.fit_transform(df_x_train[['daily_bill']])
df_x_train['account.Charges.Total'] = std_scaler.fit_transform(df_x_train[['account.Charges.Total']])
df_x_train['customer.tenure'] = mm_scaler.fit_transform(df_x_train[['customer.tenure']])

df_x_train.head()


,account.Contract_Month-to-month,account.Contract_One year,account.Contract_Two year,account.PaymentMethod_Bank transfer,account.PaymentMethod_Credit card,account.PaymentMethod_Electronic check,account.PaymentMethod_Mailed check,customer.gender,customer.SeniorCitizen,customer.Partner,customer.Dependents,customer.tenure,account.Charges.Total,daily_bill
0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.125000,-0.356879,0.201725
1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.125000,-0.414711,0.029929
2,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.055556,-0.838713,0.321904
3,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.180556,0.117556,0.944821
4,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.041667,-0.870294,0.806875


In [9]:
# Verify NaNs
print(df_x_train.isna().sum()) 

# Verify Infinits
print(np.isinf(df_x_train).sum())

account.Contract_Month-to-month            0
account.Contract_One year                  0
account.Contract_Two year                  0
account.PaymentMethod_Bank transfer        0
account.PaymentMethod_Credit card          0
account.PaymentMethod_Electronic check     0
account.PaymentMethod_Mailed check         0
customer.gender                            0
customer.SeniorCitizen                     0
customer.Partner                           0
customer.Dependents                        0
customer.tenure                            0
account.Charges.Total                     11
daily_bill                                11
dtype: int64
account.Contract_Month-to-month           0
account.Contract_One year                 0
account.Contract_Two year                 0
account.PaymentMethod_Bank transfer       0
account.PaymentMethod_Credit card         0
account.PaymentMethod_Electronic check    0
account.PaymentMethod_Mailed check        0
customer.gender                           0
custo

In [10]:
# 1. Identificar os índices das linhas que NÃO possuem NaN em X
valid_indexes = df_x_train.dropna().index

# 2. Filtrar ambos os DataFrames usando esses índices (garante alinhamento perfeito)
df_x_train = df_x_train.loc[valid_indexes]
df_y_train = df_y_train.loc[valid_indexes]

In [11]:
const_x_train = sm.add_constant(df_x_train)
model_statsmodels = sm.Logit(df_y_train, const_x_train, hasconst=True).fit()
print(model_statsmodels.summary())

Optimization terminated successfully.
         Current function value: 0.420674
         Iterations 9
                           Logit Regression Results                           
Dep. Variable:                  Churn   No. Observations:                 7256
Model:                          Logit   Df Residuals:                     7242
Method:                           MLE   Df Model:                           13
Date:                Thu, 05 Feb 2026   Pseudo R-squ.:                  0.2626
Time:                        08:53:44   Log-Likelihood:                -3052.4
converged:                       True   LL-Null:                       -4139.6
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                             coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------
const                                     -1.2335   

In [12]:
final_columns = [
    'const',
    'account.Contract_Month-to-month', 
    'account.Contract_One year', 
    'account.PaymentMethod_Electronic check',
    'customer.SeniorCitizen',
    'customer.Dependents',
    'customer.tenure',
    'daily_bill'
]

df_x_train = const_x_train[[c for c in final_columns if c in const_x_train.columns]]

model_statsmodels = sm.Logit(df_y_train, df_x_train).fit()
print(model_statsmodels.summary())

Optimization terminated successfully.
         Current function value: 0.429523
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:                  Churn   No. Observations:                 7256
Model:                          Logit   Df Residuals:                     7248
Method:                           MLE   Df Model:                            7
Date:                Thu, 05 Feb 2026   Pseudo R-squ.:                  0.2471
Time:                        08:53:44   Log-Likelihood:                -3116.6
converged:                       True   LL-Null:                       -4139.6
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                             coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------
const                                     -1.8719   

## Targeted Analyses

In [13]:
px.box(original_dataset, x='Churn', y='customer.tenure', title='Tenure by Churn Status')

In [14]:
px.box(original_dataset, x='Churn', y='account.Charges.Total', title='Total Charges by Churn Status')

# Predictive Modeling 🎯

In [15]:
X_train, X_test, y_train, y_test = train_test_split(df_x_train, df_y_train, test_size=0.2, random_state=42, stratify=df_y_train)

In [16]:
X_train.shape

(5804, 8)

In [17]:
X_test.shape

(1452, 8)

In [18]:
y_train.shape

(5804, 1)

In [19]:
y_test.shape

(1452, 1)

## KNN model

In [20]:
k = np.sqrt(len(X_train))
k

np.float64(76.18398781896364)

In [21]:
knn = KNeighborsClassifier(n_neighbors=int(k))
knn.fit(X_train, y_train.values.ravel())

knn.score(X_test, y_test)

0.7796143250688705

In [22]:
y_pred = knn.predict(X_test)

print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f} | "
      f"Precision: {precision_score(y_test, y_pred):.4f} | "
      f"Recall: {recall_score(y_test, y_pred):.4f} | "
      f"F1-Score: {f1_score(y_test, y_pred):.4f}")

print("\nconfusion matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.7796 | Precision: 0.5957 | Recall: 0.4492 | F1-Score: 0.5122

confusion matrix:
 [[964 114]
 [206 168]]

Classification Report:
               precision    recall  f1-score   support

           0       0.82      0.89      0.86      1078
           1       0.60      0.45      0.51       374

    accuracy                           0.78      1452
   macro avg       0.71      0.67      0.68      1452
weighted avg       0.77      0.78      0.77      1452



## Random Forest Model

In [23]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train.values.ravel())

y_pred_rf = rf_model.predict(X_test)

print(f"Accuracy: {accuracy_score(y_test, y_pred_rf):.4f} | "
      f"Precision: {precision_score(y_test, y_pred_rf):.4f} | "
      f"Recall: {recall_score(y_test, y_pred_rf):.4f} | "
      f"F1-Score: {f1_score(y_test, y_pred_rf):.4f}")

print("\nConfusion Matrix (RF):\n", confusion_matrix(y_test, y_pred_rf))
print("\nClassification Report:\n", classification_report(y_test, y_pred_rf))

Accuracy: 0.7645 | Precision: 0.5500 | Recall: 0.4706 | F1-Score: 0.5072

Confusion Matrix (RF):
 [[934 144]
 [198 176]]

Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.87      0.85      1078
           1       0.55      0.47      0.51       374

    accuracy                           0.76      1452
   macro avg       0.69      0.67      0.68      1452
weighted avg       0.75      0.76      0.76      1452

